In [1]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [ ]:
## wikitq: 0.491 second/query 
## tab_fact: 0.29 second/query

In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from time import sleep
import argparse
from vllm.lora.request import LoRARequest
import json
from tqdm import tqdm
import os 

INFO 10-22 01:36:40 [__init__.py:216] Automatically detected platform cuda.


In [3]:
os.environ['VLLM_ATTENTION_BACKEND'] = 'FLASH_ATTN'

In [3]:
llm_path = '/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507'
# llm_path = '/data/workspace/yanmy/models/Qwen2.5-7B-Instruct/'
llm = LLM(model=llm_path,
        tensor_parallel_size=2,
        # enforce_eager=True,
        enable_chunked_prefill=True,
        max_model_len=23000,
        # num_scheduler_steps=10,
        enable_prefix_caching=True,
        gpu_memory_utilization = 0.85,
        max_num_seqs=256,
        # kv_cache_dtype='fp8'
        )

tokenizer = AutoTokenizer.from_pretrained(llm_path)

INFO 10-20 22:36:24 [utils.py:233] non-default args: {'max_model_len': 23000, 'tensor_parallel_size': 2, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.85, 'max_num_seqs': 256, 'disable_log_stats': True, 'enable_chunked_prefill': True, 'model': '/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507'}
INFO 10-20 22:36:24 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-20 22:36:24 [model.py:1510] Using max model len 23000


INFO 10-20 22:36:24 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=1870096) INFO 10-20 22:36:25 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=1870096) INFO 10-20 22:36:25 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507', speculative_config=None, tokenizer='/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=23000, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(EngineCore_DP0 pid=1870096) (Worker_TP1 pid=1870120) INFO 10-20 22:36:29 [default_loader.py:267] Loading weights took 0.91 seconds
(EngineCore_DP0 pid=1870096) (Worker_TP0 pid=1870118) INFO 10-20 22:36:29 [default_loader.py:267] Loading weights took 0.95 seconds
(EngineCore_DP0 pid=1870096) (Worker_TP1 pid=1870120) INFO 10-20 22:36:29 [gpu_model_runner.py:2653] Model loading took 3.8657 GiB and 1.087936 seconds
(EngineCore_DP0 pid=1870096) (Worker_TP0 pid=1870118) INFO 10-20 22:36:29 [gpu_model_runner.py:2653] Model loading took 3.8657 GiB and 1.150435 seconds
(EngineCore_DP0 pid=1870096) (EngineCore_DP0 pid=1870096) (Worker_TP0 pid=1870118) (Worker_TP1 pid=1870120) INFO 10-20 22:36:37 [backends.py:548] Using cache directory: /home/aizoo/.cache/vllm/torch_compile_cache/6bf20df3c5/rank_0_0/backbone for vLLM's torch.compile
INFO 10-20 22:36:37 [backends.py:548] Using cache directory: /home/aizoo/.cache/vllm/torch_compile_cache/6bf20df3c5/rank_1_0/backbone for vLLM's torch.compile
(Engin

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:04<00:00, 15.44it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 20.69it/s]


(EngineCore_DP0 pid=1870096) (Worker_TP1 pid=1870120) INFO 10-20 22:37:35 [gpu_model_runner.py:3480] Graph capturing finished in 7 secs, took 1.53 GiB
(EngineCore_DP0 pid=1870096) (Worker_TP0 pid=1870118) INFO 10-20 22:37:35 [gpu_model_runner.py:3480] Graph capturing finished in 7 secs, took 1.53 GiB
(EngineCore_DP0 pid=1870096) INFO 10-20 22:37:35 [core.py:210] init engine (profile, create kv cache, warmup model) took 65.39 seconds
INFO 10-20 22:37:35 [llm.py:306] Supported_tasks: ['generate']


In [4]:
def response_vllm(llm,all_instructions,sample_num: int):
    text_all = []
    # all_instructions = prompts_list
    for prompt in tqdm(all_instructions, desc="Formatting Prompts"):
        messages = [{"role": "user", "content": prompt}]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        text_all.append(text)
    if sample_num == 1:
        sampling_params = SamplingParams(temperature=0, top_p=1, max_tokens=2048) 
    else:
        ## top_k/min_p 传不进去
        sampling_params = SamplingParams(n=sample_num, temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=2048) 
    outputs = llm.generate(text_all, sampling_params)
    generation_list = []
    for output in tqdm(outputs, desc="Processing Outputs"):
        generated_text = [o.text for o in output.outputs]
        generation_list.append(generated_text)
    return generation_list


In [3]:
## 参数记录 for wikitq
N_PARALLEL = 32 ## only for convert_df_type_parallel.py of data preprocess，一次性离线的
# llm_path = '/data/workspace/yanmy/models/Qwen3-30B-A3B-Instruct-2507-FP8'
# llm_path = '/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507'
llm_path = '/data/workspace/yanmy/models/Qwen2.5-7B-Instruct'
embedding_base_model_path = '/data/workspace/yanmy/models/bge-m3'
router_model_path = '/data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-finetuned/' ## fine-tuned router model path
check_moedl_path = '/data/workspace/yanmy/HybridRAG/H-STAR/check/output/bge-reranker-v2-m3-finetuned/' ## check model path
# tmp_save_path = 'datasets/schedule_test/wikitq' ## 临时存储路径
tmp_save_path = 'datasets/schedule_test/wikitq_api_server_7B' ## 临时存储路径
dataset_name = 'wikitq'
split = 'test'
tau = 0.82
check_tau = 0.8
ALL_LABELS = [
    'Base', 'Select_Row', 'Select_Column', 'Execute_SQL', 'RAG_20_5', 
]


In [5]:
## 参数记录 for wikitq
N_PARALLEL = 32 ## only for convert_df_type_parallel.py of data preprocess，一次性离线的
embedding_base_model_path = '/data/workspace/yanmy/models/bge-m3'
router_model_path = '/data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-finetuned/' ## fine-tuned router model path
check_moedl_path = '/data/workspace/yanmy/HybridRAG/H-STAR/check/output/bge-reranker-v2-m3-finetuned/' ## check model path
tmp_save_path = 'datasets/schedule_test/wikitq_7B' ## 临时存储路径
dataset_name = 'wikitq'
split = 'test'
tau = 0.82
check_tau = 0.8
ALL_LABELS = [
    'Base', 'Select_Row', 'Select_Column', 'Execute_SQL', 'RAG_20_5', 
]


### Pipeline 的过程分为以下调度步骤
- 所有调度需要的文件均放在了 `schedule_test/datasets/wikitq`与`schedule_test/datasets/tab_fact`文件夹里面
1. （独立的）跑下面的指令：`python convert_df_type_parallel.py --dataset_name tab_fact --split test_small --output_path datasets/schedule_test/tab_fact/wikitq_df_processed.npy --num_workers 32`
2. (独立的)读取 `datasets/schedule_test/tab_fact/wikitq_df_processed.npy`,构造 inference_router 的文件，输出到`datasets/schedule_test/tab_fact/router_query.pkl`
3. (独立的)构造 DB
4. (GPU-21GB) inference router model，输出route result 到 `datasets/schedule_test/tab_fact/inference_result.pkl` 通过命令：`python inference_router.py --input_path datasets/schedule_test/tab_fact/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn --output_path datasets/schedule_test/tab_fact/inference_result.pkl`
5. (CPU) 解析 router 结果，生成初步的路径并开始任务调度(重排)
6. (GPU-1.8G) 根据 `datasets/schedule_test/tab_fact/RAG_index.npy`，执行 RAG 任务，根据下面的指令
7. (GPU-vLLM) LLM 根据初步路径，对`Select_Row`, `Select_Column`, `Execute_SQL`执行中间命令
8. (CPU) 根据中间结果，link db 做sql command parse，执行 sql,返回 subtable_list
9. (GPU-1.8G) 加载并运行 Check Model，跑 3 次（FORWARD 机制）
10. (GPU-vLLM) add execute SQL
11. (GPU-vLLM) generate qa command, inference vLLM, evaluate


In [18]:
## step:1
f'python convert_df_type_parallel.py --dataset_name {dataset_name} --split {split} --output_path {tmp_save_path}/wikitq_df_processed.npy --num_workers {N_PARALLEL}'

'python convert_df_type_parallel.py --dataset_name wikitq --split test --output_path datasets/schedule_test/wikitq/wikitq_df_processed.npy --num_workers 32'

In [4]:
### 读取必要文件（step 2）
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'{tmp_save_path}/wikitq_df_processed.npy',allow_pickle=True).item()


In [7]:
## step 2: 构建semantic router的输入文件，并输出
semantic_router = {}
# semantic_router[
for index in range(len(dataset)):
    semantic_router[index] = {}
    semantic_router[index]['query'] = dataset[index]['question']
    semantic_router[index]['title'] = dataset[index]['table']['page_title']
    semantic_router[index]['table'] = wikitq_df_processed[index]
    label_list = []
    semantic_router[index]['label'] = label_list
with open(f'{tmp_save_path}/router_query.pkl','wb') as f:
    pickle.dump(semantic_router,f)

In [5]:
## step 3: Consturct DB
from utils.multi_db_v2 import NeuralDB, Executor
table_titles = [dataset[i]['table']['page_title'] for i in range(len(dataset))]
db = NeuralDB(tables=wikitq_df_processed, table_titles=table_titles)
executor = Executor()

Initializing NeuralDB...


Creating tables in DB:   0%|          | 0/4344 [00:00<?, ?it/s]

In [9]:
## step 4: inference router model
f'python inference_router.py --input_path {tmp_save_path}/router_query.pkl --model_path {router_model_path} --output_path {tmp_save_path}/inference_result.pkl'

'python inference_router.py --input_path datasets/schedule_test/wikitq_api_server_30B/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-finetuned/ --output_path datasets/schedule_test/wikitq_api_server_30B/inference_result.pkl'

In [6]:
## step 5
with open(f'{tmp_save_path}/inference_result.pkl', 'rb') as f:
    error_analysis_row = pickle.load(f)
# with open(f'../router/inference_results.pkl', 'rb') as f:
#     error_analysis_row = pickle.load(f)
ranked_result = process_error_analysis_list(error_analysis_row, truncate=True, tau=0.82)
# pd.DataFrame([str(r) for r in ranked_result.values()]).value_counts()


0                                             
['Execute_SQL']                                   2657
['Base']                                           927
['Select_Row', 'Select_Column', 'Execute_SQL']     267
['Select_Column', 'Select_Row', 'Execute_SQL']     220
['Select_Row', 'Execute_SQL']                       63
['RAG_20_5', 'Select_Row', 'Execute_SQL']           43
['Select_Column', 'Execute_SQL']                    32
['Select_Row', 'RAG_20_5', 'Execute_SQL']           32
['RAG_20_5', 'Execute_SQL']                         24
['Select_Column', 'RAG_20_5', 'Execute_SQL']        22
['RAG_20_5', 'Select_Column', 'Execute_SQL']        21
['RAG_20_5']                                        21
['Select_Row']                                      15

In [7]:
## step 5
## Organize LLM query list
LLM_query_list = {}
for method in ALL_LABELS:
    LLM_query_list[method] = {}
    LLM_query_list[method]['index'] = []
    LLM_query_list[method]['query'] = []
    LLM_query_list[method]['qa'] = []
for index in range(len(dataset)):
    for method in ALL_LABELS:
        if method in ranked_result[index]:
            # method_list = ranked_result[index][method]
            # method_list.append(index)
            LLM_query_list[method]['index'].extend([index])
            if method=='Select_Column':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/col_select_sql.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            elif method=='Select_Row':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/row_select_sql.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            elif method == 'Execute_SQL':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_wtq.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            # elif method == 'Base':
            #     prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/text_reason_tabfact.txt',processed=True)
            #     LLM_query_list[method]['qa'].extend([prompt])


In [12]:
## step 5 暂存 RAG 的 list，输出到路径
np.save(f'{tmp_save_path}/RAG_index.npy',LLM_query_list['RAG_20_5']['index'])

- 执行`Hybrid_Retrieve.py` 任务

In [13]:
## step 6: RAG
f"python Hybrid_Retrieve_Update_dict.py \
    --model_path {embedding_base_model_path} \
    --dataset_name {dataset_name} \
    --split {split} \
    --index_path {tmp_save_path}/RAG_index.npy \
    --output_path {tmp_save_path}/Hybrid_Retrieve_output.npy \
    --max_rows 50 \
    --max_cols 10 \
    --processed_df_path {tmp_save_path}/wikitq_df_processed.npy \
    --rewrite_query_path datasets/schedule_test/{dataset_name}/rewrite_query.npy"

'python Hybrid_Retrieve_Update_dict.py     --model_path /data/workspace/yanmy/models/bge-m3     --dataset_name wikitq     --split test     --index_path datasets/schedule_test/wikitq_api_server_30B/RAG_index.npy     --output_path datasets/schedule_test/wikitq_api_server_30B/Hybrid_Retrieve_output.npy     --max_rows 50     --max_cols 10     --processed_df_path datasets/schedule_test/wikitq_api_server_30B/wikitq_df_processed.npy     --rewrite_query_path datasets/schedule_test/wikitq/rewrite_query.npy'

In [8]:
## load Hybrid_Retrieve 的结果
RAG_20_5 = np.load(f'{tmp_save_path}/Hybrid_Retrieve_output.npy',allow_pickle=True).item()

In [9]:
## step 7: api vLLM
### Query LLM with async engine，启动模型采用 sh llm_server.sh，这个是不是可以放在调度的第一个环节，不计入时间

metrics_rows = {}
summary = {}
for method in ['Select_Row','Select_Column']:
    prompt_list = LLM_query_list[method]['query']
    response_list,metrics_rows[method],summary[method] = infer_prompts(prompt_list, temperature=0.7, top_p=0.8, sample_num=2, llm_path = llm_path)
    LLM_query_list[method]['response'] = response_list
for method in ['Execute_SQL']:
    prompt_list = LLM_query_list[method]['query']
    response_list,metrics_rows[method],summary[method] = infer_prompts(prompt_list, temperature=0.7, top_p=0.8, sample_num=3, llm_path = llm_path)
    LLM_query_list[method]['response'] = response_list

Processed prompts:   0%|          | 0/774 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 3594/3594 [14:45<00:00,  4.06it/s, est. speed input=9833.87 toks/s, output=2103.60 toks/s] 


- row_sql

In [10]:
## step 8: SQP parse row

from tqdm.notebook import tqdm
sub_table_list_all = {}
filtered_tables_row = {}
row_sql_index_list = LLM_query_list['Select_Row']['index']
row_sql_response_list = LLM_query_list['Select_Row']['response']
for i in range(len(row_sql_index_list)):
    sample_num = [0,1]
    sub_table_list = []
    for sample_index in sample_num:
        ## 相对距离
        index = row_sql_index_list[i]
        #  = row_sql_response_list[i][sample_index]
        # relevant_index = row_sql_index_list.index(index)
        original_text = row_sql_response_list[i][sample_index]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        try:
            result_1 = executor.sql_exec(sql.replace('``','`').replace("COUNT(*)", "*"), db, table_id=index)
            sub_table_list.append(pd.DataFrame(result_1['rows'],columns=result_1['header']))
        except:
            continue
    print(dataset[index]['question'],dataset[index]['answer_text'])
    # match_subtables(large_table=wikitq_df_processed[index], sub_tables=[pd.DataFrame(result_1['rows'],columns=result_1['header'])])
    # pd.DataFrame(result_1['rows'],columns=result_1['header'])
    sub_table_list_all[index] = sub_table_list
    filtered_df = retrieve_rows_by_subtables(wikitq_df_processed[index],sub_table_list)
    if len(filtered_df) == 0:
        filtered_df = wikitq_df_processed[index]
    filtered_tables_row[index] = filtered_df

Executing on table_id 4 ('w4'): SELECT * AS num_first_places FROM w WHERE placing = 1;
Executing on table_id 4 ('w4'): SELECT * AS num_first_places     FROM w     WHERE placing = 1;
what is the number of 1st place finishes across all events? ['17']
Executing on table_id 12 ('w12'): WITH relevant_rows AS (        SELECT * FROM Guillermo_García-López WHERE tournament < '2009'    )    SELECT SUM(year_end_ranking) AS total_ranking     FROM relevant_rows;
Executing on table_id 12 ('w12'): WITH relevant_years AS (        SELECT `year end ranking`         FROM Guillermo_García-López         WHERE `year end ranking` IS NOT NULL AND `tournament` < '2009'    )    SELECT SUM(`year end ranking`) AS total_ranking     FROM relevant_years;
what is the combined score of year end rankings before 2009? ['440']
Executing on table_id 15 ('w15'): WITH last_3_games AS (        SELECT * FROM w ORDER BY date DESC LIMIT 3    )    SELECT SUM(CASE WHEN SUBSTR(result, 0, INSTR(result, '-')-1) != '' THEN CAST(SUBS

- col_sql

In [11]:
## step 8: SQP parse column
filtered_tables = {}
filtered_headers = {}
col_sql_index_list = LLM_query_list['Select_Column']['index']
col_sql_response_list = LLM_query_list['Select_Column']['response']
# for ind in tqdm(range(len(wikitq_df_processed))):
for i in range(len(col_sql_index_list)):
    ind = col_sql_index_list[i]
    
    input_df = wikitq_df_processed[ind]
    response_list = col_sql_response_list[i]
    assert isinstance(response_list,list)
    filtered_table, final_headers = filter_dataframe_from_responses(response_list, input_df, add_row_id=True)
    filtered_tables[ind] = filtered_table
    filtered_headers[ind] = final_headers

- SQL-exec

In [12]:
## step 8: SQP parse execute
from tqdm.notebook import tqdm
sample_num = 3
sql_exec_df = {}
valid_parse = 0
warning_list = []
sql_executable_count = []
exec_sql_index_list = LLM_query_list['Execute_SQL']['index']
exec_sql_response_list = LLM_query_list['Execute_SQL']['response']
# for index in tqdm(range(len(wikitq_df_processed))):
for i in range(len(exec_sql_index_list)):
    index = exec_sql_index_list[i]
    sql_exec_df[index] = []
    for sample_ind in range(sample_num):
        
        # original_text = eval(wikitq_df_output.iloc[index,-1])[sample_ind]
        original_text = exec_sql_response_list[i][sample_ind]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        if [index,sample_ind] in warning_list:
            sql = ''
            print(index,sample_ind)
        if sql!='':
            # print(result)
            try:
                result = executor.sql_exec(sql.replace('``','`'), db, table_id=index,add_row_id = True)
                df = pd.DataFrame(result['rows'],columns=result['header'])
            except:
                df = pd.DataFrame()
        else:
            df = pd.DataFrame()
        sql_exec_df[index].append(df)
        if(len(df)>0):
            valid_parse+=1
            sql_pair = {}
            sql_pair['id'] = index
            sql_pair['sample_ind'] = sample_ind
            sql_pair['sql'] = sql
            sql_pair['table'] = df
            sql_executable_count.append(sql_pair)
sql_exec_df_output = merge_clean_and_format_df_dict(sql_exec_df)
    # print(original_text)

Executing on table_id 0 ('w0'): SELECT SUBSTR(cyclist, INSTR(cyclist, ' (') + 3, INSTR(cyclist, ')') - INSTR(cyclist, ' (') - 3) AS country, COUNT(*) AS num_cyclists FROM w WHERE rank BETWEEN 1 AND 10 GROUP BY country ORDER BY num_cyclists DESC LIMIT 1;
Executing on table_id 0 ('w0'): SELECT SUBSTR(cyclist, INSTR(cyclist, ' ') + 1) AS country, COUNT(*) AS num_cyclists FROM w WHERE rank BETWEEN 1 AND 10 GROUP BY country ORDER BY num_cyclists DESC LIMIT 1;
Executing on table_id 0 ('w0'): SELECT SUBSTR(cyclist, INSTR(cyclist, ' ') + 1) AS nationality, COUNT(*) AS cyclist_count FROM w WHERE rank <= 10 GROUP BY nationality ORDER BY cyclist_count DESC LIMIT 1;
Executing on table_id 1 ('w1'): SELECT `1940/41` FROM w WHERE `description losses` = 'murdered';
Executing on table_id 1 ('w1'): SELECT `1940/41` FROM w WHERE `description losses` = 'murdered';
Executing on table_id 1 ('w1'): SELECT `1940/41` FROM w WHERE `description losses` = 'murdered';
Executing on table_id 3 ('w3'): SELECT origina

/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

[   alert_name  backspace_name
0  bell (bel)  backspace (bs),   unicode name    unicode name
0   bell (bel)  backspace (bs)]


/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

[  result
0     No,                                          via
0  hainault barkingside, gants hill , ilford
1     woodford green, snaresbrook , wanstead,   result result
0     No     No]


/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

[   year       playoffs  prev_playoffs
0  1976  league champs  league champs,           manager  year  year
0  roy hartsfield  1975  1976,    first_year  second_year
0        1975         1976]


/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

In [13]:
## step 8: SQP parse aggregate
processed_table = {} ## processed_table包括了inference LLM后所有结果的聚合,key 与 method_key 保持一致
processed_table['Base'] = wikitq_df_processed
processed_table['Select_Row'] = filtered_tables_row
processed_table['Select_Column'] = filtered_tables
processed_table['RAG_20_5'] = RAG_20_5
processed_table['Execute_SQL'] = sql_exec_df_output ## SQL聚合结果！
processed_table['Execute_SQL_count'] = sql_executable_count ## SQL单个结果，用作 CrossEncoder 跨模态查询

In [15]:
## step 8: SQP parse aggregate
np.save(f'{tmp_save_path}/processed_table.npy',processed_table)

In [40]:
processed_table = np.load(f'{tmp_save_path}/processed_table.npy',allow_pickle=True).item()

- Run Check Model

In [14]:
## Step 9: generated check model data 
# 对每一条数据，维护一个 Sequence_Operator
# 每条数据应该对应一个 Dict,这个Dict 包含了3 个参数：
    # id: index
    # Sequence: Operator Sequence
    # Terminated: True/False
    # Check Score: CrossEncoder 分数
    # Check Status: True/False,当前是否被计算
# 对每条数据，如果 Terminated = True，则不参与计算
# 对应的，如果 Terminated = False，那么查看 Check Status,如果 Statue=False，那么计算 Check_Score
# 每一轮计算之后，Check_Score 更新，Check Statue更新，该轮开始判定，用一个 threshold
# 如果 Check Status = True, Check Score >= Threshold，那么标注 Terminated = True，当前路径被接收
# 如果 Check Status = True, Check Score < Threshold，那么Terminated = False, Check Status = False, Check Score 清空
# Check Score 触发 RollBack 机制，即除了‘Execute_SQL'外，Extract 回退，重新计算当前流程
# 如果当前操作符只有'Execute_SQL’和空（所有 RollBack 均未被接收），那么触发 FORWARD,Terminated  = True，强制修改 Operator Sequence 为['Execute_SQL']
Check_Model_Data_Sequence = {}
for key in ranked_result.keys(): ## 遍历所有数据,初始化
    start_sequence = ranked_result[key]
    Check_Model_Data_Sequence[key] = {}
    Check_Model_Data_Sequence[key]['id'] = key
    Check_Model_Data_Sequence[key]['Sequence'] = start_sequence
    if start_sequence == ['Base'] or start_sequence == ['Execute_SQL']: ## Terminated 不参与计算
        Check_Model_Data_Sequence[key]['Terminated'] = True
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
    else:
        Check_Model_Data_Sequence[key]['Terminated'] = False
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
# Check_Model_Data_Sequence[4]
for key in Check_Model_Data_Sequence.keys():
    data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table)
    Check_Model_Data_Sequence[key]['data_entry'] = data_entry
## Check
    

- load ReRanking Model

In [15]:
## step 9 load rerank model
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

In [16]:
## step 9: iterative check
check_tau = check_tau
for loop in range(3):
    updated_data = batch_rerank_scores(reranker_model, Check_Model_Data_Sequence, batch_size=16)
    Check_Model_Data_Sequence = updated_data
    ## Check Terminal Status
    for key in Check_Model_Data_Sequence.keys():
        if Check_Model_Data_Sequence[key]['Terminated'] == True:
            continue
        else:
            if Check_Model_Data_Sequence[key]['Check_Status'] == True:
                if Check_Model_Data_Sequence[key]['Check_Score'] >= check_tau: ## 0.5 threshold
                    Check_Model_Data_Sequence[key]['Terminated'] = True
                else:
                    Check_Model_Data_Sequence[key]['Terminated'] = False
                    Check_Model_Data_Sequence[key]['Check_Status'] = False
                    Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
                    current_sequence = Check_Model_Data_Sequence[key]['Sequence']
                    ROLLBACK_seq, terminated_flag = ROLLBACK(current_sequence)
                    Check_Model_Data_Sequence[key]['Sequence'] = ROLLBACK_seq
                    Check_Model_Data_Sequence[key]['Terminated'] = terminated_flag
                    if terminated_flag == False:
                        data_entry = Prepare_Data_for_Operator_Sequence(key,ROLLBACK_seq,dataset, processed_table)
                        Check_Model_Data_Sequence[key]['data_entry'] = data_entry
            else:
                raise ValueError("Check Status should be True when Terminated is False after reranking.")

Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 862 items with batch size 16...


initial target device:   0%|          | 0/2 [00:00<?, ?it/s]/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
initial target device:  50%|█████     | 1/2 [00:05<00:05,  5.53s/it]/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Chunks: 100%|██████████| 2/2 [00:04<00:00,  2.28s/it]


Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 115 items with batch size 16...


Chunks: 100%|██████████| 2/2 [00:00<00:00,  7.66it/s]

Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
No items to process.


In [17]:
## step 9 load rerank model
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

### Check Additional Execute SQL and Execute

In [18]:
## step 10: add missing SQL
SQL_list_final = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        SQL_list_final.append(index)
add_sql_list = list(set(SQL_list_final) - set(LLM_query_list['Execute_SQL']['index']))
add_sql_query_list = []
for index in add_sql_list:
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_wtq.txt',processed=True)
    add_sql_query_list.extend([prompt])
# add_sql_response_list = response_vllm(llm,add_sql_query_list, sample_num=3)
add_sql_response_list = infer_prompts(add_sql_query_list, sample_num=3, temperature=0.7, top_p=0.8,llm_path=llm_path)[0]

Processed prompts: 100%|██████████| 6/6 [00:12<00:00,  2.09s/it, est. speed input=1901.46 toks/s, output=362.62 toks/s]


In [19]:
### step 10: add sql parse and execute
from tqdm.notebook import tqdm
sample_num = 3
sql_exec_df = {}
valid_parse = 0
warning_list = []
sql_executable_count = []
# exec_sql_index_list = LLM_query_list['Execute_SQL']['index']
# exec_sql_response_list = LLM_query_list['Execute_SQL']['response']
# for index in tqdm(range(len(wikitq_df_processed))):
for i in range(len(add_sql_list)):
    index = add_sql_list[i]
    sql_exec_df[index] = []
    for sample_ind in range(sample_num):
        
        # original_text = eval(wikitq_df_output.iloc[index,-1])[sample_ind]
        original_text = add_sql_response_list[i][sample_ind]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        if [index,sample_ind] in warning_list:
            sql = ''
            print(index,sample_ind)
        if sql!='':
            # print(result)
            try:
                result = executor.sql_exec(sql.replace('``','`'), db, table_id=index,add_row_id = True)
                df = pd.DataFrame(result['rows'],columns=result['header'])
            except:
                df = pd.DataFrame()
        else:
            df = pd.DataFrame()
        sql_exec_df[index].append(df)
        if(len(df)>0):
            valid_parse+=1
            sql_pair = {}
            sql_pair['id'] = index
            sql_pair['sample_ind'] = sample_ind
            sql_pair['sql'] = sql
            sql_pair['table'] = df
            sql_executable_count.append(sql_pair)
sql_exec_df_output_new = merge_clean_and_format_df_dict(sql_exec_df)
    # print(original_text)
for index in sql_exec_df_output_new.keys():
    sql_exec_df_output[index] = sql_exec_df_output_new[index]
processed_table['Execute_SQL'] = sql_exec_df_output

Executing on table_id 2018 ('w2018'): WITH ranked_designers AS (     SELECT          year,         designer(s) AS current_designer,         LAG(designer(s)) OVER (ORDER BY year) AS previous_designer     FROM          w ) SELECT DISTINCT      current_year.year AS year_1,      current_year.current_designer,      next_year.year AS year_2,      next_year.previous_designer FROM      ranked_designers AS current_year JOIN      ranked_designers AS next_year ON current_year.year = next_year.year - 1 WHERE      current_year.current_designer = next_year.previous_designer;
Executing on table_id 2018 ('w2018'): SELECT w1.year, w1.`designer(s)`, w2.`designer(s)`  FROM w AS w1  JOIN w AS w2  ON w1.year + 1 = w2.year  AND w1.`designer(s)` = w2.`designer(s)`;
Executing on table_id 2700 ('w2700'): SELECT city, passengers FROM w WHERE city != 'quintana roo, cancún' ORDER BY passengers DESC LIMIT 1;
Executing on table_id 2700 ('w2700'): SELECT city, passengers FROM w WHERE city != 'quintana roo, cancún' O

/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

In [20]:
np.save(f'{tmp_save_path}/Check_Model_Data_Sequence.npy',Check_Model_Data_Sequence)

In [21]:
## step 8: SQP parse aggregate
np.save(f'{tmp_save_path}/processed_table.npy',processed_table)

In [5]:
Check_Model_Data_Sequence = np.load(f'{tmp_save_path}/Check_Model_Data_Sequence.npy',allow_pickle=True).item()

In [6]:
processed_table = np.load('datasets/schedule_test/wikitq_train_free/processed_table.npy',allow_pickle=True).item()

In [22]:
## step 11: generate QA result
prompt_list = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    prompt = build_wikitq_prompt_from_df(dataset,Check_Model_Data_Sequence[index]['data_entry']['table'],index,template_path='../prompts/text_reason_wtq.txt',processed=True)
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        # try:
        evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
        prompt = prompt + evidence
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list

In [23]:
## step 11: execute QA
# qa_final = response_vllm(llm,prompt_list, sample_num=1)
qa_final = infer_prompts(prompt_list,temperature=0, top_p=1, sample_num=1,llm_path=llm_path)
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final[0]
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final[0]]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

Processed prompts: 100%|██████████| 4344/4344 [06:05<00:00, 11.88it/s, est. speed input=26306.47 toks/s, output=1187.33 toks/s]


  0%|          | 0/4344 [00:00<?, ?it/s]

Correct Samples: 2860; Total Samples: 4344
Accuracy: 65.84


In [58]:
wikitq_df = pd.DataFrame(dataset).iloc[:100]
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final[0]
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final[0]]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

Correct Samples: 80; Total Samples: 100
Accuracy: 80.00


In [28]:
wikitq_df.to_csv('wikitq_test_result_30B.csv')

In [39]:
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df.iloc[:100], dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

Correct Samples: 82; Total Samples: 100
Accuracy: 82.00


In [46]:
## step 11: execute QA
qa_final = response_vllm(llm,prompt_list, sample_num=1)
qa_final = 
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final

Formatting Prompts:   0%|          | 0/4344 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/4344 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4344 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

Processing Outputs:   0%|          | 0/4344 [00:00<?, ?it/s]

In [47]:
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

  0%|          | 0/4344 [00:00<?, ?it/s]

Correct Samples: 2847; Total Samples: 4344
Accuracy: 65.54


In [9]:
## step 11: execute QA
# qa_final = response_vllm(llm,prompt_list, sample_num=1)
qa_final = infer_prompts(prompt_list,temperature=0, top_p=1, sample_num=1,llm_path = llm_path)
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final[0]
## step 12: evaluate
wikitq_df['predict'] = [str(s) for s in qa_final[0]]
acc_all, error_index_all, format_error_index_all = evaluate_predictions(dataset_name, wikitq_df, dataset)

Processed prompts:  17%|█▋        | 723/4344 [01:36<08:03,  7.48it/s, est. speed input=15665.28 toks/s, output=1542.73 toks/s]


KeyboardInterrupt: 